In [ ]:
from web3 import Web3
import json
import time

w3 = Web3(Web3.HTTPProvider(f'https://api.securerpc.com/v1'))

def generate_transaction_payload(action, contract_address, abi, params, from_address, private_key, value=0):
    """
    Generates and sends a transaction payload for the specified action.

    :param action: The action to perform ('swap', 'bridge', 'lend').
    :param contract_address: The address of the contract to interact with.
    :param abi: The ABI of the contract.
    :param params: A dictionary of parameters required for the contract method.
    :param from_address: The address initiating the transaction.
    :param private_key: The private key of the from_address for signing the transaction.
    :param value: The amount of Ether (in wei) to send with the transaction.
    :return: Transaction receipt.
    """
    contract = w3.eth.contract(address=contract_address, abi=abi)

    if action == 'swap':
        function_name = 'swapExactTokensForTokens'  
        txn = contract.functions.swapExactTokensForTokens(
            params['amount_in'],
            params['amount_out_min'],
            params['path'],
            params['to'],
            params['deadline']
        ).build_transaction({
            'from': from_address,
            'value': value,
            'nonce': w3.eth.get_transaction_count(from_address),
            'gas': params.get('gas', 200000),
            'gasPrice': w3.to_wei('10', 'gwei')
        })

    elif action == 'bridge':
        raise NotImplementedError("Bridge action is not implemented in this test script.")

    elif action == 'lend':
        raise NotImplementedError("Lend action is not implemented in this test script.")

    else:
        raise ValueError("Invalid action specified.")

    signed_txn = w3.eth.account.sign_transaction(txn, private_key=private_key)


    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)

    print(f"Transaction sent with hash: {tx_hash.hex()}")

    
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    print(f"Transaction receipt: {tx_receipt}")

    return tx_receipt


In [ ]:

import json
import time
from web3 import Web3
from decimal import Decimal


def build_transaction_payload(
    action,
    web3,
    abi_path,
    contract_address,
    from_address,
    private_key,
    params
):
    """
    Builds transaction payloads for different actions.
    :param action: The action to perform ('swap', 'bridge', 'lend')
    :param web3: Web3 instance connected to a network
    :param abi_path: Path to the ABI file
    :param contract_address: Address of the contract
    :param from_address: Your wallet address
    :param private_key: Private key for signing transactions
    :param params: Dictionary of parameters required for the action
    :return: Raw transaction data ready to be signed and sent
    """

    
    with open(abi_path, 'r') as abi_file:
        contract_abi = json.load(abi_file)

    
    contract = web3.eth.contract(address=contract_address, abi=contract_abi)

    
    nonce = web3.eth.get_transaction_count(from_address)
    gas_price = web3.eth.gas_price  
    chain_id = web3.eth.chain_id    

    if action == 'swap':
        
        token_in = params['token_in']
        token_out = params['token_out']
        amount_in = params['amount_in']
        amount_out_min = params['amount_out_min']
        to_address = params['to_address']
        deadline = params.get('deadline', int(time.time()) + 600)  
        txn = contract.functions.swapExactTokensForTokens(
            amount_in,
            amount_out_min,
            [token_in, token_out],
            to_address,
            deadline
        ).build_transaction({
            'from': from_address,
            'nonce': nonce,
            'gasPrice': gas_price,
            'gas': 250000,  
            'chainId': chain_id,
        })

    elif action == 'lend':
        raise NotImplementedError("Lend action is not implemented yet.")
    elif action == 'bridge':
        raise NotImplementedError("Bridge action is not implemented yet.")
    else:
        raise ValueError(f"Unsupported action: {action}")
    signed_txn = web3.eth.account.sign_transaction(txn, private_key=private_key)
    return signed_txn.raw_transaction.hex() #return txn


def test_swap_on_sepolia():
    
    web3 = Web3(Web3.HTTPProvider('https://1rpc.io/sepolia'))

    
    from_address = web3.to_checksum_address('0x10EdEdD9Fa10D711F56d8451e8336aA2Bcd95517')
    private_key = '5b6afd833cebca9ccc3a553d2da59149db91d9b30756b009183f0a3a604f72de'     

    
    params = {
        'token_in': web3.to_checksum_address('0xFFf9976782D46Cc05630d1F6eBaB18B2324D6b14'),      
        'token_out': web3.to_checksum_address('0x7Fc21CEB0C5003576Ab5E101EB240C2B822C95D2'),    
        'amount_in': web3.to_wei(0.01, 'ether'),  
        'amount_out_min': 10,                 
        'to_address': from_address,          
        
    }

    
    abi_path = 'abis/IUniswapV2Router02.json'  
    contract_address = web3.to_checksum_address('0xeE567Fe1712Faf6149d80dA1E6934E354124CfE3')

    
    raw_txn = build_transaction_payload(
        action='swap',
        web3=web3,
        abi_path=abi_path,
        contract_address=contract_address,
        from_address=from_address,
        private_key=private_key,
        params=params
    )

    
    tx_hash = web3.eth.send_raw_transaction(raw_txn)
    print(f"Transaction sent with hash: {web3.to_hex(tx_hash)}")

    
    tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    print(f"Transaction receipt: {tx_receipt}")

if __name__ == '__main__':
    test_swap_on_sepolia()


Transaction sent with hash: 0x2b1685d69e2f9b6204421bf701e099f375b36aa9eeb081cddbe480db477d3d07
Transaction receipt: AttributeDict({'blockHash': HexBytes('0x6ed8e49e7461ef04127e7a5b59deb170503368f3d99f4bbe13a43b167cc45899'), 'blockNumber': 7089116, 'contractAddress': None, 'cumulativeGasUsed': 10954585, 'effectiveGasPrice': 917997864, 'from': '0x10EdEdD9Fa10D711F56d8451e8336aA2Bcd95517', 'gasUsed': 27840, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 0, 'to': '0xeE567Fe1712Faf614

In [ ]:
import json
import time
from web3 import Web3

def bridge_layer_zero(
    web3,
    endpoint_contract_address,
    abi_path,
    from_address,
    private_key,
    dst_chain_id,
    destination_address,
    payload,
    gas_amount,
    gas_price,
    zro_payment_address=None,
    adapter_params=b""
):
    """
    Bridges tokens/messages using LayerZero.
    
    :param web3: Web3 instance
    :param endpoint_contract_address: Address of LayerZero endpoint contract
    :param abi_path: Path to LayerZero Endpoint ABI file
    :param from_address: Address initiating the bridge transaction
    :param private_key: Private key of the sender
    :param dst_chain_id: Destination chain ID (LayerZero ID)
    :param destination_address: Destination contract address (bytes format)
    :param payload: The payload to send to the destination contract
    :param gas_amount: Gas amount for transaction
    :param gas_price: Gas price for transaction
    :param zro_payment_address: Optional ZRO payment address for gas (default: None)
    :param adapter_params: Optional adapter parameters (default: empty bytes)
    """
    # Load ABI
    with open(abi_path, 'r') as abi_file:
        contract_abi = json.load(abi_file)

    # Initialize contract
    contract = web3.eth.contract(address=web3.to_checksum_address(endpoint_contract_address), abi=contract_abi)

    # Build transaction
    nonce = web3.eth.get_transaction_count(from_address)
    chain_id = web3.eth.chain_id
    txn = contract.functions.send(
        dst_chain_id,
        destination_address,
        payload,
        from_address,
        zro_payment_address or "0x0000000000000000000000000000000000000000",  # Default ZRO payment address
        adapter_params
    ).build_transaction({
        'from': from_address,
        'nonce': nonce,
        'gas': gas_amount,
        'gasPrice': gas_price,
        'chainId': chain_id,
        'value': web3.to_wei(0.01, 'ether')  # Adjust based on fees
    })

    # Sign and send transaction
    signed_txn = web3.eth.account.sign_transaction(txn, private_key=private_key)
    tx_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
    print(f"Transaction sent: {web3.to_hex(tx_hash)}")

    # Wait for receipt
    receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    print(f"Transaction receipt: {receipt}")
    return receipt


# Example usage
def test_bridge_on_sepolia():
    web3 = Web3(Web3.HTTPProvider('https://1rpc.io/sepolia'))
    
    from_address = web3.to_checksum_address('0xYourWalletAddressHere')
    private_key = '0xYourPrivateKeyHere'
    endpoint_contract_address = '0xEndpointContractAddressHere'

    # Load destination contract address and payload
    destination_address = Web3.to_bytes(hexstr="0xDestinationContractHere")
    payload = b"Test payload"

    abi_path = 'abis/LayerZeroEndpoint.json'

    receipt = bridge_layer_zero(
        web3=web3,
        endpoint_contract_address=endpoint_contract_address,
        abi_path=abi_path,
        from_address=from_address,
        private_key=private_key,
        dst_chain_id=101,  # Example: Destination chain ID
        destination_address=destination_address,
        payload=payload,
        gas_amount=300000,
        gas_price=web3.eth.gas_price
    )
    print(f"Bridge completed with receipt: {receipt}")


if __name__ == '__main__':
    test_bridge_on_sepolia()
